In [ ]:
import sys
import os
import pandas as pd
import MetaTrader5 as mt5

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from services.mt5_data_service import MT5DataService
from services.indicator_service import IndicatorService

print("Libraries and services imported successfully.")

print("\nAttempting to initialize connection to MT5 terminal...")
if not mt5.initialize():
    print("FATAL: initialize() failed, error code =", mt5.last_error())
    quit()

print("SUCCESS: Connection to MT5 terminal established.")

Libraries and services imported successfully.

Attempting to initialize connection to MT5 terminal...
SUCCESS: Connection to MT5 terminal established.


In [ ]:
data_svc = MT5DataService()

SYMBOL = "EURUSD"
TIMEFRAME = 'H1'
START_DATE = "2020-01-01"

print(f"\n--- Fetching Primary Asset Data ({SYMBOL}) ---")
historical_df = data_svc.get_all_historical_data(symbol=SYMBOL, timeframe_str=TIMEFRAME, start_date=START_DATE)

if historical_df is not None:
    print(f"\nSUCCESS: {SYMBOL} data fetched successfully.")
    print(f"Total H1 Candles Fetched: {len(historical_df)}")
    print(f"Data Range: from {historical_df.index.min()} to {historical_df.index.max()}")
else:
    print("\nERROR: Failed to fetch data. Please check the logs.")


--- Fetching Primary Asset Data (EURUSD) ---
MT5DataService (Hist): Fetching all data for EURUSD on H1 since 2020-01-01...
MT5DataService (Hist): Successfully downloaded and processed 35422 candles.
MT5DataService (Hist): Data range from 2020-01-02 03:00:00+00:00 to 2025-09-05 06:00:00+00:00

SUCCESS: EURUSD data fetched successfully.
Total H1 Candles Fetched: 35422
Data Range: from 2020-01-02 03:00:00+00:00 to 2025-09-05 06:00:00+00:00


In [ ]:
if historical_df is not None and not historical_df.empty:
    
    indicator_svc = IndicatorService()
    
    print("\n--- Calculating the 'Ultimate' suite of indicators for EURUSD ---")
    
    features_df = indicator_svc.add_all_indicators(historical_df)
    
    if features_df is not None and not features_df.empty:
        print("\nSUCCESS: All features successfully calculated.")
        print(f"Final DataFrame shape: {features_df.shape}")
        print(f"Data Range: from {features_df.index.min()} to {features_df.index.max()}")
        print("\nData Head with all features:")
        display(features_df.head())
    else:
        print("\nERROR: Feature calculation resulted in an empty DataFrame.")
else:
    print("\nSkipping feature engineering due to data fetching errors in the previous cell.")
    features_df = None

IndicatorService: Initialized.

--- Calculating the 'Ultimate' suite of indicators for EURUSD ---
IndicatorService: Calculating the final, optimized suite of indicators...
IndicatorService: Final, optimized indicator suite successfully added.

SUCCESS: All features successfully calculated.
Final DataFrame shape: (35197, 30)
Data Range: from 2020-01-14 10:00:00+00:00 to 2025-09-04 04:00:00+00:00

Data Head with all features:


,open,high,low,close,volume,ichimoku_senkou_span_a,ichimoku_senkou_span_b,ichimoku_tenkan_sen,ichimoku_kijun_sen,ichimoku_chikou_span,...,BBB_20_2.0,BBP_20_2.0,ATRr_14,ADX_14,DMP_14,DMN_14,SQZ_20_2.0_20_1.5,SQZ_ON,SQZ_OFF,SQZ_NO
time,,,,,,,,,,,,,,,,,,,,,
2020-01-14 10:00:00+00:00,1.11338,1.11345,1.11262,1.11303,1847,1.111788,1.111045,1.11353,1.112980,1.11489,...,0.116049,0.004413,0.000695,35.164893,18.919802,16.165903,-0.000030,1,0,0
2020-01-14 11:00:00+00:00,1.11303,1.11303,1.11194,1.11206,1732,1.111738,1.111045,1.11319,1.113165,1.11522,...,0.171165,-0.301387,0.000723,33.453425,16.882456,21.142960,-0.000405,0,1,0
2020-01-14 12:00:00+00:00,1.11206,1.11207,1.11050,1.11102,3061,1.111738,1.111045,1.11247,1.112595,1.11521,...,0.260943,-0.332919,0.000784,33.685720,14.466117,31.244421,-0.000918,0,1,0
2020-01-14 13:00:00+00:00,1.11102,1.11233,1.11042,1.11217,3034,1.111738,1.111045,1.11243,1.112555,1.11595,...,0.275196,0.114706,0.000864,33.384978,14.331335,26.310714,-0.001137,0,1,0
2020-01-14 14:00:00+00:00,1.11217,1.11232,1.11136,1.11220,2609,1.111765,1.111045,1.11243,1.112555,1.11575,...,0.286510,0.164538,0.000871,33.105717,13.202861,24.238963,-0.001320,0,1,0


In [ ]:
if 'features_df' in locals() and features_df is not None and not features_df.empty:
    
    data_dir = '../../data'
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    output_filename = "eurusd_h1_features.csv"
    output_path = os.path.join(data_dir, output_filename)
    
    features_df.to_csv(output_path)

    print(f"\nSUCCESS: Processed H1 feature data for EURUSD saved to: '{output_path}'")
else:
    print("\nERROR: No valid data to save. Please check the output of the previous cells.")

if 'mt5' in locals() and mt5.terminal_info():
    mt5.shutdown()
    print("\nConnection to MT5 terminal has been shut down. Notebook complete.")
else:
    print("\nNotebook complete (MT5 connection was already closed or failed to initialize).")


SUCCESS: Processed H1 feature data for EURUSD saved to: '../../data\eurusd_h1_features.csv'

Connection to MT5 terminal has been shut down. Notebook complete.
